In [1]:
pip install pymysql

     -------------------------------------- 43.8/43.8 kB 533.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import csv
import pymysql

In [3]:
# 创建游标对象
cursor = conn.cursor()

In [5]:
# 读取csv文件
with open('apagon_data_processed.csv', 'r', encoding='utf-8', newline='') as f:
    read = csv.reader(f)

    user_list = []                   # Create a  new user list for input
    for row in read:
        if any(row):                 # Pick up the non-blank row of list
            user_list.append(row)    # Compose all the rest data into the list

    
    # 一行一行地存，除去第一行和第一列
    for each in user_list[1:]:
        # print(each)
        i = tuple(each[1:])        
        # 使用SQL语句添加数据
        print(str(i))
        sql = "INSERT INTO py_test VALUES" + str(i) # db_top100是表的名称
        cursor.execute(sql) #执行SQL语句
    
    conn.commit() # 提交数据
    cursor.close() # 关闭游标
    conn.close() # 关闭数据库


('1', '1', '18.357271999999998', '-65.721699000000001', 'false', '2022-08-18 00:41:43.796974 +00:00', '8/18/2022 0:41:43', 'Sabana', 'Luquillo', '7210773114', '973', '1')


OperationalError: (1136, "Column count doesn't match value count at row 1")

In [8]:
pip install sqlalchemy

     ---------------------------------------- 1.9/1.9 MB 1.9 MB/s eta 0:00:00
     ------------------------------------- 192.2/192.2 kB 11.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# -*- coding:UTF-8 -*-

import pymysql
import pandas as pd
from sqlalchemy import create_engine

# 数据库信息
mysql_setting = {
    'host': 'localhost',
    'port': 3306,
    'user': 'root',
    'passwd': '',
    # 数据库名称
    'db': 'py_sql',
    'charset': 'utf8'
}
# 表名
# 如果不存在表，则自动创建
table_name = 'py_test'
# 文件路径
path = 'apagon_data_processed.csv'

data = pd.read_csv(path, encoding='utf-8')
# print(data)
engine = create_engine(
    "mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}".format(**mysql_setting), max_overflow=5)
data.to_sql(table_name, engine, index=False, if_exists='replace',)
print('导入成功...')


导入成功...


In [1]:
import pymysql

In [2]:
conn = pymysql.connect(host='localhost', user='root',
                       password='', db='py_sql', charset='utf8')


In [15]:
# fail test

with conn.cursor() as cursor:
    # 3. 通过游标对象向数据库发出SQL语句获取执行结果
    cursor.execute('select a.OID_,a.latitude,a.longitude from py_test as a ,py_test as b where a.latitude=b.latitude and a.longitude=b.longitude and (a.barrio!=b.barrio or a.municipio!=b.municipio)')
    # 4. 通过游标抓取数据
    for dept in cursor.fetchall():
        print(dept)
        # print(f'{dept["name"]}\t{dept["location"]}')
conn.close()
print('done')

done


In [3]:
final_list=[]
with conn.cursor() as cursor:
    # 3. 通过游标对象向数据库发出SQL语句获取执行结果
    cursor.execute('select * from py_test where OID_ in (select max(OID_) from py_test group by latitude,longitude,timestamp)')
    # 4. 通过游标抓取数据
    for dept in cursor.fetchall():
        print(dept)
        final_list.append(dept)
        # print(f'{dept["name"]}\t{dept["location"]}')
conn.close()
print('done')
# SELECT  *  FROM  TABLE  WHERE  ID  IN  ( SELECT  MAX (ID)  FROM  TABLE  GROUP  BY  [去除重复的字段名列表,....])

(1, 1, 1, 18.357272, -65.721699, 0, '2022-08-18 00:41:43.796974 +00:00', '8/18/2022 0:41:43', 'Sabana', 'Luquillo', 7210773114, 973, 1)
(2, 2, 2, 18.327498, -65.658402, 0, '2022-08-18 00:50:11.652021 +00:00', '8/18/2022 0:50:11', 'Barrio Pueblo', 'Fajardo', 7205932350, 131, 1)
(3, 1, 3, 18.383904, -66.073697, 0, '2022-08-18 04:26:59.675919 +00:00', '8/18/2022 4:26:59', 'Monacillo Urbano', 'San Juan', 7212753936, 848, 1)
(4, 1, 4, 18.383904, -66.073697, 1, '2022-08-18 04:27:26.451067 +00:00', '8/18/2022 4:27:26', 'Monacillo Urbano', 'San Juan', 7212753936, 848, 1)
(5, 1, 5, 18.296937, -65.635969, 0, '2022-08-18 07:47:17.258636 +00:00', '8/18/2022 7:47:17', 'Quebrada Vueltas', 'Fajardo', 7205367868, 918, 1)
(6, 1, 6, 18.296937, -65.635969, 0, '2022-08-18 07:54:14.670663 +00:00', '8/18/2022 7:54:14', 'Quebrada Vueltas', 'Fajardo', 7205367868, 918, 1)
(7, 1, 7, 18.296937, -65.635969, 0, '2022-08-18 07:59:49.331342 +00:00', '8/18/2022 7:59:49', 'Quebrada Vueltas', 'Fajardo', 7205367868, 918

In [4]:
len(final_list)

5747

In [5]:
import pandas as pd

In [6]:
data_new=pd.DataFrame(final_list)
data_new.columns=['OID_','Join_Count','TARGET_FID','latitude','longgitude','has_power','timestamp','date','barrio','municipio','geoid','barro_id','report_count']

In [7]:
data_new.head(10)

,OID_,Join_Count,TARGET_FID,latitude,longgitude,has_power,timestamp,date,barrio,municipio,geoid,barro_id,report_count
0,1,1,1,18.357272,-65.721699,0,2022-08-18 00:41:43.796974 +00:00,8/18/2022 0:41:43,Sabana,Luquillo,7210773114,973,1
1,2,2,2,18.327498,-65.658402,0,2022-08-18 00:50:11.652021 +00:00,8/18/2022 0:50:11,Barrio Pueblo,Fajardo,7205932350,131,1
2,3,1,3,18.383904,-66.073697,0,2022-08-18 04:26:59.675919 +00:00,8/18/2022 4:26:59,Monacillo Urbano,San Juan,7212753936,848,1
3,4,1,4,18.383904,-66.073697,1,2022-08-18 04:27:26.451067 +00:00,8/18/2022 4:27:26,Monacillo Urbano,San Juan,7212753936,848,1
4,5,1,5,18.296937,-65.635969,0,2022-08-18 07:47:17.258636 +00:00,8/18/2022 7:47:17,Quebrada Vueltas,Fajardo,7205367868,918,1
5,6,1,6,18.296937,-65.635969,0,2022-08-18 07:54:14.670663 +00:00,8/18/2022 7:54:14,Quebrada Vueltas,Fajardo,7205367868,918,1
6,7,1,7,18.296937,-65.635969,0,2022-08-18 07:59:49.331342 +00:00,8/18/2022 7:59:49,Quebrada Vueltas,Fajardo,7205367868,918,1
7,8,1,8,18.362455,-65.624783,0,2022-08-18 11:40:13.435489 +00:00,8/18/2022 11:40:13,Cabezas,Fajardo,7205309732,207,1
8,9,1,9,18.226750,-66.603683,0,2022-08-18 14:20:16.939891 +00:00,8/18/2022 14:20:16,Jayuya Abajo,Jayuya,7207338542,1258,1
9,10,1,10,18.323669,-66.366251,0,2022-08-18 15:26:20.739466 +00:00,8/18/2022 15:26:20,Padilla,Corozal,7204757677,808,1


In [37]:
from sqlalchemy import create_engine
# 数据库信息
mysql_setting = {
    'host': 'localhost',
    'port': 3306,
    'user': 'root',
    'passwd': '',
    # 数据库名称
    'db': 'py_sql',
    'charset': 'utf8'
}
# 表名
# 如果不存在表，则自动创建
table_name = 'py_final'
# print(data)
engine = create_engine(
    "mysql+pymysql://{user}:{passwd}@{host}:{port}/{db}".format(**mysql_setting), max_overflow=5)
data_new.to_sql(table_name, engine, index=False, if_exists='replace',)
print('导入成功...')


导入成功...


In [8]:
pip install openyxl

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement openyxl (from versions: none)
ERROR: No matching distribution found for openyxl


In [8]:
data_new.to_csv('final.csv',index=None)